### 2D/3D Gaussian pulse

In [1]:
from Electromagnetics.FDTD import *
import numpy as np
import time
import matplotlib.pyplot as plt

n_dim = 3
r0, r1, dr = [None]*3
S = 0.9
n_t = None
save_every = 10

r_c = None
omega = 2.0*np.pi
s_pml = None
d_pml = None

eps_r_0 = 1.0
eps_r = 1.0
eps_r_max = max(eps_r_0, eps_r)

L_0 = 2.0*np.pi/omega
L_g = L_0/eps_r

if n_dim==3:
    r0 = np.array([0.0, 0.0, 0.0])
    r1 = np.array([1.0, 1.0, 1.0])*10.0*L_g
    dr = np.array([1.0, 1.0, 1.0])*L_g/20
    S /= np.sqrt(3)
    n_t = 200
    
    r_c = (r0+r1)/2.0    
    d_pml = np.array([1.0, 1.0, 1.0])*L_g
    s_pml = np.array([1.0+1.0j, 1.0+1.0j, 1.0+1.0j])/d_pml*3.0
elif n_dim==2:
    r0 = np.array([0.0, 0.0])
    r1 = np.array([1.0, 1.0])*10.0*L_g
    dr = np.array([1.0, 1.0])*L_g/30
    S /= np.sqrt(2)
    n_t = 2000
    
    r_c = (r0+r1)/2.0    
    d_pml = np.array([1.0, 1.0])*L_g
    s_pml = np.array([1.0+1.0j, 1.0+1.0j])/d_pml*3.0

elif n_dim==1:
    assert False
    
dt = S*dr[0]/np.sqrt(eps_r_max)
d_v = np.prod(dr)

print('dr : ', dr)
print('dt: ', dt)
print('S: ', S)

dr :  [ 0.05  0.05  0.05]
dt:  0.0259807621135
S:  0.519615242271


In [2]:

fdtd = FDTDSimulator(vbose=True)

fdtd.SetSimulationBox(r0, r1, dr, dt)
fdtd.save_every = save_every 

d_x_ = np.zeros(n_dim)
d_x_[0] = (r1[0]-r0[0])
d_x_pml = np.zeros(n_dim)
d_x_pml[0] = d_pml[0]

T_per = 2.0*np.pi/omega
t0_j = 2.0*T_per
T_j = 1.0*T_per
f_t = lambda t: np.exp(-(t - t0_j)**2/T_j**2)*np.cos(omega*t)*(t<=T_j) + np.cos(omega*t)*(t>T_j)

## J_e
W_G = 4*L_0
r_je = r_c.copy()
r_je[0] = np.floor(r_je[0]/dr[0])*dr[0]
r_je[1] = np.floor(r_je[1]/dr[1])*dr[1]+dr[1]/2
print('ind : ', np.floor(r_je[0]/dr[0]))
src_dir_e = 'y'
je_type = FVTypes.JeSheet
je_mag = None
if n_dim==2:
    je_mag = lambda r: 1.0/dr[0]*np.exp(-(r[1]-r_je[1])**2/W_G**2)
elif n_dim==3:
    je_mag = lambda r: 1.0/dr[0]*np.exp(-((r[1]-r_je[1])**2+(r[2]-r_je[2])**2)/W_G**2)
norm_dir = 'x'
je_args = {'r0':r_je, 'mag':je_mag, 'f_t':f_t, 'src_dir':src_dir_e, 'norm_dir':norm_dir}

fdtd.AddSources({'type':je_type, 'name':'Je', 'args':je_args})

## J_m
W_G = L_0
r_jm = r_je.copy()
r_jm[0] = r_je[0] - dr[0]/2
src_dir_m = 'z'
jm_type = FVTypes.JmSheet
jm_mag = None
if n_dim==2:
    jm_mag = lambda r: 1.0/dr[0]*np.exp(-(r[1]-r_jm[1])**2/W_G**2)
elif n_dim==3:
    jm_mag = lambda r: 1.0/dr[0]*np.exp(-((r[1]-r_jm[1])**2+(r[2]-r_jm[2])**2)/W_G**2)
norm_dir = 'x'
jm_args = {'r0':r_jm, 'mag':jm_mag, 'f_t':lambda t: f_t(t-dt/2), 'src_dir':src_dir_m, 'norm_dir':norm_dir}

fdtd.AddSources({'type':jm_type, 'name':'Jm', 'args':jm_args})


eps_args = {'r0':r0, 'r1':r1, 'mag_in':eps_r, 'mag_out':eps_r_0}
fdtd.AddMaterial({'type':FVTypes.EpsIsoBox, 'name':'eps', 'args':eps_args})


pml_args = {'d_pml':d_pml, 's_pml':s_pml}
fdtd.AddPML({'type':FVTypes.UPML, 'name':'pml', 'args':pml_args})

set_eps_out = False

r_out = r_c

if n_dim==3:
    fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_out, 'args':{'A':'E', 'A_dir':'y', 'O_dir':'y', 'name':'Ey'}})
elif n_dim<=2:
    fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_out, 'args':{'A':'E', 'A_dir':'x', 'O_dir':None, 'name':'Ex'}})
    fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_out, 'args':{'A':'E', 'A_dir':'y', 'O_dir':None, 'name':'Ey'}})
    fdtd.AddViewPlane({'type':FVTypes.VPSide, 'r':r_out, 'args':{'A':'E', 'A_dir':'z', 'O_dir':None, 'name':'Ez'}})

        
fdtd.StepFields(n_t)
n_saved = fdtd.n_saved
    


ind :  100.0
self.E[0].shape: (200, 201, 201)
self.D[0].shape: (200, 201, 201)
self.B[0].shape: (201, 200, 200)
self.H[0].shape: (201, 200, 200)
self.Je[0].shape: (200, 201, 201)
sides inds_min_x: (99, 100, 100) d: 0.0353553390593
sides inds_min_y: (100, 100, 100) d: 0.0
sides inds_min_z: (100, 100, 99) d: 0.0353553390593
self.Jm[0].shape: (201, 200, 200)
face inds_min_x: (99, 100, 99) d: 0.0353553390593
face inds_min_y: (99, 100, 99) d: 0.0353553390593
face inds_min_z: (99, 100, 100) d: 0.0
self.eps[0].shape: (200, 201, 201)
sides inds_min_x: (99, 100, 100) d: 0.025
sides inds_min_y: (100, 99, 100) d: 0.025
sides inds_min_z: (100, 100, 99) d: 0.025
self.ViewPlanes:  dict_keys(['Ey'])
self.ChunkInd: None dt: 0.0259807621135 n_t: 200 t_final: 5.19615242271
starting time:  20272.281049107
self.upml_dF_D[0].shape: (200, 201, 201)
self.upml_F_D[0].shape: (200, 201, 201)
self.upml_dF_H[0].shape: (201, 200, 200)
self.upml_F_H[0].shape: (201, 200, 200)
has_pml: True
finishing time:  21042.704

In [3]:
import matplotlib.pyplot as plt

r_E, E_out_list = fdtd.GetOutputs('Ey')
x_E, y_E, z_E = [None]*3
if n_dim==3:
    x_E, y_E, z_E = r_E
elif n_dim==2:
    x_E, y_E = r_E
elif n_dim==1:
    x_E = r_E[0]
e_max = [np.max(np.abs(e)) for e in E_out_list]
print('max(e_max):', max(e_max))
e_max = max(e_max)
    

print(x_E.shape, E_out_list[0].shape)

import matplotlib.pyplot as plt
import matplotlib.animation as animation


Saved outputs: dict_keys(['Ey'])
max(e_max): 1.01922326609
(201, 201) (201, 201)


In [4]:
%matplotlib tk

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib

# Set up formatting for the movie files
#Writer = animation.writers['mencoder']    ## avconv ffmpeg mencoder
#writer = Writer(fps=15, metadata=dict(artist='Me'))
#writer = animation.FFMpegWriter()

plt.rcParams["figure.figsize"] = (5, 5)

plt_field = 'E'
save_anim = False
file_format = 'gif'   ## 'gif' / 'mp4'

logscale=False
log_0 = np.log(e_max)-20

use_subplots = False

font = {'family' : 'serif', 'weight' : 'normal', 'size' : 14}
matplotlib.rc('font', **font)

fig = None
if n_dim>=2:
    def animate_E(i):
        plt.clf()
        #fig = plt.pcolor(y_E, z_E, E_out_list[i])
        if not logscale:
            #fig = plt.imshow(np.abs(E_out_list[i]).T, vmin=0.0, vmax=e_max/100)
            fig = plt.imshow(E_out_list[i].T, origin='lower', vmin=-e_max/10, vmax=e_max/10, cmap='rainbow', \
                             aspect='auto')
        else:
            E_log = np.log(np.abs(E_out_list[i]))
            E_log = (E_log>=log_0)*E_log + (E_log<log_0)*(log_0)
            fig = plt.imshow(E_log.T, origin='lower', vmin=log_0, vmax=np.log(e_max), cmap='rainbow', \
                             aspect='auto')

        #CB = plt.colorbar(fig, shrink=0.8, extend='both')
        ax = plt.gca()
        ax.set_adjustable('box-forced')
        ax.axes.get_xaxis().set_ticks([])
        ax.axes.get_yaxis().set_ticks([])
        plt.colorbar()
        if i%10==0:
            print("{}/{}".format(i, n_saved), end=" ")
        return fig

    fig = plt.figure()
    axes = plt.gca()
    anim = None
    if plt_field=='E':
        anim = animation.FuncAnimation(fig, animate_E, frames=n_saved, interval=1, repeat=False)
    else:
        raise ValuError()

    if save_anim:
        if file_format=='gif':
            anim.save('other/e-2d.gif', writer="imagemagick", fps=15, dpi=200)
        elif file_format=='mp4':
            anim.save('other/e-2d.mp4', writer="mencoder", fps=15, dpi=200)
    plt.show()

    

0/20 0/20 10/20 